## Background

### Pauli matrices

### Stabilizer groups

Let $G$ be a group and $X$ be a set. A **group action** $\varphi$ of $G$ on $X$ is a function $\varphi \colon G \times X \to X$ such that:
-  for all $x \in X$, we have $\varphi(\mathrm{id}_G, x) = x$;
-  for all $g,h \in G$, we have $\varphi(gh,x) = \varphi(g,\varphi(h,x))$.

Often we drop the notation $\varphi$ and simply write $gx$ to mean $\varphi(g,x)$.
The **stabilizer** of an element $x \in X$ with respect to the group action of $G$ on $X$ is the set:
\begin{equation*}
    G_x = \{ g \in G | gx = x \}.
\end{equation*}
That is, the elements of $G$ that act like the identity on $x$. Notice that $G_x$ is closed under composition and inverses:
- For elements $g$ and $h$ in $G$, we have that
\begin{equation*}
    ghx = g(hx) = g(x) = x.
\end{equation*}
- For an element $g$ in $G$, we have that
\begin{equation*}
    g^{-1}x = g^{-1}(gx) = \mathrm{id}_Gx = x.
\end{equation*}

So $G_x$ is a subgroup of $G$.

#### Example 1: Permutation groups.


Consider the symmetric group $S_3$ acting on the set $X = \{0,1,2\}$ via permutations. Each permutation $\sigma \in S_3$ can be written using integers $a_0$, $a_1$ and $a_2$ such that $\{a_0,a_1,a_2\} = \{0,1,2\}$ by assigning
\begin{equation*}
    0 \to a_0, \;\; 1 \to a_1, \; \; 2 \to a_2.
\end{equation*}
We can represent this more compactly by a tuple $\sigma = (a_0,a_1,a_2)$ where the $i^{\text{th}}$ entry is the image of $i$ under the permutation $\sigma$. For example, the tuple $(2,1,0)$ represents the permutation $0 \to 2$, $1 \to 1$, and $2 \to 0$. Using this representation we can create a Python class that represents a permutation.

In [ ]:
class Permutation:
    """
    A permutation of {0, ..., n-1}, represented as a tuple A = (a0, a1, ..., an-1)
    where i -> A[i].
    """
    def __init__(self,
                 mapping: tuple[int,...]):
        self.n = len(mapping)
        if set(mapping) != set(range(self.n)):
            raise ValueError("A permutation must contain each number from 0 to n-1 exactly once.")
        self.mapping = mapping

    def __call__(self, x: int):
        """
        Returns the image of x under the permutation.
        Can be called like a function: i.e. p = Permutation(), then p(x) = p.__call__(x).
        """
        if not (x in range(self.n)):
            raise ValueError(f"Permutation only acts on integers 0-n. Got x={x}.")
        return self.mapping[x]
    
    def compose(self, other: 'Permutation') -> 'Permutation':
        """
        Return the composition of two permutations: C = B ∘ A.
        For permutations A = self and B = other, the composition C(i) = B(A(i)).
        """
        if other.n != self.n:
            raise ValueError(f"Permutations of different sets cannot be composed. First is on {self.n}, second is on {other.n}.")
    
        return Permutation(tuple(other(self(i)) for i in range(self.n)))
    
    def fixes(self, i: int) -> bool:
        """
        Returns True if the permutation fixes i and False otherwise.
        """
        if not (0 <= i < self.n):
            raise ValueError(f"Invalid i. Permutation acts on 0 to n-1. Got {i}.")
        return self(i) == i

    def fixed_points(self) -> list[bool]:
        """
        Returns a list of booleans indicating which points are fixed by the permutation.
        The ith entry is True if the permutation fixes the element i, and False otherwise.
        """
        return [self(i) == i for i in range(self.n)]
    
    def __repr__(self):
        return f"Permutation {self.mapping}"

We wish to consider stabilizers of points in $X$. We first define the elements of the symmetric group $S_3$.

In [ ]:
S3_elements = [(0,1,2),
               (0,2,1),
               (1,0,2),
               (1,2,0),
               (2,0,1),
               (2,1,0)]

S3 = [Permutation(p) for p in S3_elements]

print(f"S3 has elements:")
for p in S3:
    print(p)

Now we can see how a permutation acts on the set $\{0,1,2\}$. Let us again take the permutation $(2,1,0)$.

In [ ]:
p = S3[-1]
print(f"{p} acts as:")
for x in range(3):
    print(f"{x} -> {p(x)}")

We can also compute the fixed points of the permutation.

In [ ]:
for x in range(3):
    print(f"{p} fixes {x}: {p.fixes(x)}")

This permutation fixes $1$, but it is not the only permutation to do so. We compute the stabilizer group $(S_3)_1$.

In [ ]:
def stabilizer(G: list[Permutation], x: int) -> list[Permutation]:
    """
    Returns the elements in the stabilizer group G_x.
    That is, the subgroup of G consisting of elements that fix x.
    """
    if not G:
        raise ValueError(f"Invalid group. Must contain at least one permutation.")
    if not (0 <= x < G[0].n):
        raise ValueError(f"Invalid x. Must be between 0 and {G[0].n-1}. Got {x}.")
    return [p for p in G if p.fixes(x)]

print(f"Elements of the stabilizer group of 1 wrt S3 action:")
for p in stabilizer(S3,1):
    print(p)

#### TODO: Understand why the quantum world can be modelled by complex Hilbert space.

#### Example 2: Pauli matrices.


The Pauli matrices are a set of three $2\times 2$ matrices in complex space given by the following:
\begin{equation*}
    X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad
    Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad
    Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}.
\end{equation*}

In [ ]:
from dataclasses import dataclass
import numpy as np

@dataclass(frozen=True)
class Pauli:
    """
    Represents a Pauli matrix
    TODO: Should the identity be here ...?
    """
    type: str

    def __post_init__(self):
        if self.type not in ["X", "Y", "Z", "Id"]:
            raise ValueError(f"There are four Pauli matrices X, Y, Z, Id. Got {self.type}.")
        
    @property
    def matrix(self):
        if self.type == "X":
            matrix = np.array([[0,1],[1,0]], dtype=complex)
        if self.type == "Y":
            matrix = np.array([[0,-1j],[1j,0]], dtype=complex)
        if self.type == "Z":
            matrix = np.array([[1,0],[0,-1]], dtype=complex)
        if self.type == "Id":
            matrix = np.eye(2, dtype=complex)
        return matrix

pauli = [Pauli("X"), Pauli("Y"), Pauli("Z"), Pauli("Id")]

for M in pauli:
    print(f"Pauli matrix {M.type}")
    print(M.matrix)

The _Pauli group_ $\mathcal{P}$ is the subgroup of complex matrices with respect to matrix multiplication that is generated by the Pauli matrices. More explicitly, we have
\begin{equation*}
    \mathcal{P} = \{\; \lambda \mathrm{Id}, \lambda X, \lambda Y, \lambda Z \; | \; \lambda = \pm 1, \pm i \;\}.
\end{equation*}

In [ ]:
coefficients = [complex(1), complex(-1), complex(0,1), complex(0,-1)]

def complex_to_int_str(z: complex) -> str:
    """
    Return a minimal string representation of a complex number with integer parts.
    - If the number is 0, returns "0".
    - If purely real, returns the integer real part.
    - If purely imaginary, returns the integer imaginary part with 'j'.
    - Otherwise, returns "a+bj" with integer real and imaginary parts.
    """
    if z == 0:
        return "0"
    elif z.imag == 0:
        return f"{int(z.real)}"
    elif z.real == 0:
        return f"{int(z.imag)}j"
    else:
        return f"{int(z.real)}+{int(z.imag)}j"

@dataclass(frozen=True)
class PauliGroupElt:
    """
    Represents an element of the Pauli group.
    """
    coefficient: complex
    pauli: Pauli

    def __post_init__(self):
        if self.coefficient not in coefficients:
            raise ValueError(f"Pauli group element has coefficient in [1,-1,i,-i]. Got {self.coefficient}.")

    @property
    def matrix(self):
        return self.coefficient * self.pauli.matrix
    
    @property
    def print_matrix(self):
        """
        Print the matrix with minimal integer representations of the complex elements.
        """
        rows, cols = self.matrix.shape
        M = []
        for row in range(rows):
            row_list = []
            for col in range(cols):
                entry = self.matrix[row, col]
                row_list.append(complex_to_int_str(entry))
            M.append(row_list)
        for r in M:
            print(r)

    @property
    def type(self):
        return self.pauli.type

    def __repr__(self):
        return f"Pauli group element {complex_to_int_str(self.coefficient)}{self.type}"

    
pauli_group = [PauliGroupElt(c, p) for c in coefficients for p in pauli]

for g in pauli_group:
    print(g)
    g.print_matrix

There is a natural action by matrices on vectors by matrix multiplication. In particular, the Pauli group acts on two dimensional complex vectors. Explicity, there is a group action of $G = \mathcal{P}$ on the set $X = \mathbb{C}^2$. Thus, we can consider stabilizer groups of sets of vectors in $X = \mathbb{C}^2$.

Fix a vector $v \in \mathbb{C}^2$ and consider the stabilizer group
\begin{equation*}
    \mathcal{P}_{v} = \{ P \in \mathcal{P} | Pv = v\}.
\end{equation*}
Notice that a matrix $M$ lies in the stabilizer $\mathcal{P}_v$ if and only if $v$ is an eigenvector of $M$ with eigenvalue $+1$. Therefore, we may find the fixed points of elements in the Pauli group by finding eigenvalues and eigenvectors.

In [ ]:
def eig_pauli(p : PauliGroupElt) -> tuple:
    M = p.matrix
    return np.linalg.eig(M)

for p in pauli_group:
    vals, vecs = eig_pauli(p)
    print(f"{p} has:")
    for idx in range(len(vals)):
        print(f"  {idx+1}. Eigenvalue = {vals[idx]:.2f} with eigenvector = {vecs[:,idx]}.")


In [ ]:
def fixed_pts(p : PauliGroupElt) -> list:
    vals, vecs = eig_pauli(p)
    fixed = []
    for idx in range(len(vals)):
        if np.isclose(vals[idx], 1.0):
            fixed.append(vecs[:,idx])
    return fixed

for p in pauli_group:
    pts = fixed_pts(p)
    if pts:
        vec_strs = [[f"{v.real:.2f}+{v.imag:.2f}j" for v in vec] for vec in pts]
        num_pts = len(pts)
        if num_pts == 1:
            print(f"{p} stabilizes 1 vector: {vec_strs[0]}.")
        else:
            print(f"{p} stabilizes {num_pts} vectors: {vec_strs}.")



Notice that every Pauli matrix stabilizes one vector. We have:
\begin{equation*}
    X \left[ \frac{1}{\sqrt(2)} \begin{pmatrix} 1 \\ 1 \end{pmatrix} \right] = \frac{1}{\sqrt(2)} \begin{pmatrix} 1 \\ 1 \end{pmatrix}, \quad
    Y \left[ \frac{1}{\sqrt(2)}i \begin{pmatrix} 1 \\ 1 \end{pmatrix} \right] = \frac{1}{\sqrt(2)}i \begin{pmatrix} 1 \\ 1 \end{pmatrix}, \quad
    Z \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \end{pmatrix}.
\end{equation*}

Therefore, a two dimensional complex vector $v$ has a non-trivial stabilizer $\mathcal{P}_v$ if and only if $v$ is one of the following:
\begin{equation*}
    \frac{1}{\sqrt(2)} \begin{pmatrix} 1 \\ 1 \end{pmatrix}, \quad
    \frac{1}{\sqrt(2)}i \begin{pmatrix} 1 \\ 1 \end{pmatrix}, \quad
    \begin{pmatrix} 1 \\ 0 \end{pmatrix}.
\end{equation*}

## Quantum error correction

The power of quantum computing comes from generalising the notion of a classical bit to a _qubit_. A classical bit is a system with a binary attribute, such as a light switch that is either off (0) or on (1). At any given time, the system occupies exactly one of the two states, 0 or 1. 

A qubit differs crucially from a classical bit. It is not a piece of information that lives in a discrete state 0 or 1, but one that can exist as a linear combination of the two states and only under measurement does in live in one of the states 0 or 1. In particular, a qubit is a physical system with a measurement operator such that the following hold:
- Prior to measurement the state of the system can be represented by a unit vector in two-dimensional complex space.
- When measured the system yields one of two binary outcomes.

Note from a mathematical point of view, a classical bit is simply a special case of a qubit that is always confined to the states 0 and 1.

#### Example: Spin states

The Stern-Gerlach experiment passes atoms through a special magnetic field and their resulting positions detected on a screen. The atoms accumulate in exactly two distinct locations, conventionally referred to as _spin up_ and _spin down_. That is, after measurement the experiment has two possible outcomes. Therefore, this system appears to be analogous to a classical bit. However, there is one fundamental difference.

The crucial difference between a bit and a qubit is what happens _before_ measurement. In the classical situation, the light switch is always either on or off even when it is unobserved. In contrast, prior to being detected on the screen, an atom is not necessarily in a _spin up_ or _spin down_ state. Instead, it is in a state that combines both possibilities and only upon measurement does the atom collapse to one of the two outcomes. More precisely, it is a linear combination of the two states spin up and spin down. Mathematically, this is notion is familiar: a state is a vector. In particular, the state is a directional vector in three-dimensional space. Interactions with other physical objects, like the electromagnetic fields, change this state continuously, for example by rotating it. But it's value is not known until it is measured.



#### Quantum errors



### Stabilizer codes

### CSS codes

### Homological codes